In [1]:
from tensorflow.keras.optimizers import RMSprop

Using TensorFlow backend.


In [2]:
from keras.preprocessing.image import ImageDataGenerator

In [3]:
import cv2

In [4]:
from keras.models import Sequential

In [5]:
from keras.layers import Conv2D, Input, ZeroPadding2D, BatchNormalization, Activation, MaxPooling2D, Flatten, Dense,Dropout

In [6]:
from keras.models import Model, load_model

In [7]:
from keras.callbacks import TensorBoard, ModelCheckpoint

In [8]:
from sklearn.metrics import f1_score

In [9]:
from sklearn.utils import shuffle

In [10]:
import imutils
import numpy as np

In [11]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense

In [12]:
import tensorflow
from tensorflow.keras.layers import Conv2D, Flatten, MaxPooling2D, Dropout
from keras.layers import BatchNormalization

In [13]:
model = tensorflow.keras.Sequential([
    Conv2D(100, (3,3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2,2),
    
    Conv2D(100, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    
    Flatten(),
    Dropout(0.5),
    Dense(50, activation='relu'),
    Dense(2, activation='softmax')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [14]:
TRAINING_DIR = r"E:\mep\Dataset\train"
train_datagen = ImageDataGenerator(rescale=1.0/255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')
train_generator = train_datagen.flow_from_directory(TRAINING_DIR, 
                                                    batch_size=10, 
                                                    target_size=(150, 150))
VALIDATION_DIR = r"E:\mep\Dataset\test"
validation_datagen = ImageDataGenerator(rescale=1.0/255)
validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR, 
                                                         batch_size=10, 
                                                         target_size=(150, 150))

Found 1315 images belonging to 2 classes.
Found 194 images belonging to 2 classes.


In [15]:
from tensorflow.keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint('model2-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')

In [16]:
history = model.fit_generator(train_generator,
                              epochs=10,
                              validation_data=validation_generator,
                              callbacks=[checkpoint])

C:\Users\user\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1963: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
132/132 [==============================] - 137s 1s/step - loss: 0.4852 - acc: 0.7392 - val_loss: 0.1817 - val_acc: 0.9175
INFO:tensorflow:Assets written to: model2-001.model\assets
Epoch 2/10
132/132 [==============================] - 132s 1s/step - loss: 0.3420 - acc: 0.8722 - val_loss: 0.1406 - val_acc: 0.9536
INFO:tensorflow:Assets written to: model2-002.model\assets
Epoch 3/10
132/132 [==============================] - 136s 1s/step - loss: 0.2635 - acc: 0.8981 - val_loss: 0.1103 - val_acc: 0.9536
INFO:tensorflow:Assets written to: model2-003.model\assets
Epoch 4/10
132/132 [==============================] - 190s 1s/step - loss: 0.2889 - acc: 0.8821 - val_loss: 0.0820 - val_acc: 0.9588
INFO:tensorflow:Assets written to: model2-004.model\assets
Epoch 5/10
132/132 [==============================] - 154s 1s/step - loss: 0.2615 - acc: 0.8966 - val_loss: 0.0943 - val_acc: 0.9639
Epoch 6/10
132/132 [==============================] - 147s 1s/step - loss: 0.2187 - acc: 0.9217 - v

In [17]:
model.save('desktop/model-010.h5')

In [28]:
pip install h5py==2.10.0 –force-reinstall

Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: '–force-reinstall'


In [18]:
import cv2
import numpy as np
from keras.models import load_model
model=load_model("desktop/model-010.h5")

results={0:'without mask',1:'mask'}
GR_dict={0:(0,0,255),1:(0,255,0)}

rect_size = 4
cap = cv2.VideoCapture(0) 


haarcascade = cv2.CascadeClassifier(r'E:\mep\haarcascade_frontalface_default.xml')

while True:
    (rval, im) = cap.read()
    im=cv2.flip(im,1,1) 

    
    rerect_size = cv2.resize(im, (im.shape[1] // rect_size, im.shape[0] // rect_size))
    faces = haarcascade.detectMultiScale(rerect_size)
    for f in faces:
        (x, y, w, h) = [v * rect_size for v in f] 
        
        face_img = im[y:y+h, x:x+w]
        rerect_sized=cv2.resize(face_img,(150,150))
        normalized=rerect_sized/255.0
        reshaped=np.reshape(normalized,(1,150,150,3))
        reshaped = np.vstack([reshaped])
        result=model.predict(reshaped)

        
        label=np.argmax(result,axis=1)[0]
      
        cv2.rectangle(im,(x,y),(x+w,y+h),GR_dict[label],2)
        cv2.rectangle(im,(x,y-40),(x+w,y),GR_dict[label],-1)
        cv2.putText(im, results[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)

    cv2.imshow('LIVE',   im)
    key = cv2.waitKey(10)
    
    if key == 27: 
        break

cap.release()

cv2.destroyAllWindows()

AttributeError: 'str' object has no attribute 'decode'